In [184]:
import pandas as pd
import numpy as np
import time
from datetime import timedelta,datetime

In [1]:
def data_split(df,well_id_int=6110299100,param_id_list=[188,200],package_size=1, leftcut=0,):
    df=df.query('well_id == @well_id_int and param_id in @param_id_list')
    del df['well_id']
    # Устанавливаем tm_time как индекс
    df.set_index('tm_time', inplace=True)

    # Отсортируем по индексу
    df.sort_index(inplace=True)

    # Преобразование с помощью pivot
    df = df.pivot(columns='param_id', values='tm_value')

    # убираем верхний индекс param_id
    df.columns.rename(None, inplace=True)

    """ Интерполяция временного ряда до частоты 1 секунда. :param df: Исходный DataFrame с временными метками и значениями :return: Новый DataFrame с интервалом в 1 секунду """
    # Интерполирование к частоте в 1 секунду
    df = df.resample('1s').mean().interpolate(method='linear', limit_direction='both')
    return df

In [186]:
def interpolate_to_1_second(df):
    """ Интерполяция временного ряда до частоты 1 секунда. :param df: Исходный DataFrame с временными метками и значениями :return: Новый DataFrame с интервалом в 1 секунду """
    interpolated_df = df.resample('1s').mean().interpolate(method='linear', limit_direction='both')
    return interpolated_df

In [4]:
def package_split(df, package_size=1, leftcut=0):
    """ Разбивает временной ряд на пакеты заданного размера. :param df: Pandas Series или DataFrame с временными метками :param package_size: Размер каждого пакета (количество точек) :param leftcut: срез слева :return: Список пакетов (каждый пакет является DataFrame) """
    packages = []
    current_packet = pd.DataFrame()  # Изначально пустой пакет
    num_points = len(df)

    # Идем по ряду с шагом package_size
    for i in range(0, num_points, package_size):
        # Берем кусок данных длиной package_size
        package = df[i:i + package_size]

         # Добавляем новые точки к текущему пакету
        current_packet = pd.concat([current_packet[leftcut:], package])

        # Добавляем пакет в список
        packages.append(current_packet)

    return packages


In [179]:
# Пример DataFrame
df = pd.DataFrame({
    'values': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'timestamp': pd.date_range(start="2023-01-01", periods=10, freq="s")
})

# Установка индекса по временным меткам
df.set_index('timestamp', inplace=True)


df=package_split(df,package_size=4,leftcut=5)
df

[                     values
 timestamp                  
 2023-01-01 00:00:00       1
 2023-01-01 00:00:01       2
 2023-01-01 00:00:02       3
 2023-01-01 00:00:03       4,
                      values
 timestamp                  
 2023-01-01 00:00:04       5
 2023-01-01 00:00:05       6
 2023-01-01 00:00:06       7
 2023-01-01 00:00:07       8,
                      values
 timestamp                  
 2023-01-01 00:00:08       9
 2023-01-01 00:00:09      10]

In [252]:
# Пример данных
data = {
    ('param_id', ''): ['188', '200'],
    ('tm_time', '2024-04-01 00:00:03'): [30.0, 74.0],
    ('tm_time', '2024-04-01 00:00:33'): [30.0, 75.0],
    ('tm_time', '2024-04-01 00:01:03'): [30.0, 74.0]
}

# Создание DataFrame
df = pd.DataFrame(data)

# Удаление первого уровня индекса столбцов
#df.columns = df.columns.droplevel(0)

# Результат
print(df)

  param_id             tm_time                                        
           2024-04-01 00:00:03 2024-04-01 00:00:33 2024-04-01 00:01:03
0      188                30.0                30.0                30.0
1      200                74.0                75.0                74.0
